In [ ]:
pip show iesta

In [1]:

import iesta.machine_learning.dataloader
#from .autonotebook import tqdm as notebook_tqdm
import iesta.stats.significance
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
import seaborn as sns

sns.reset_defaults()

sns.set(
    rc={'figure.figsize':(3,2)}, 
    #style="white" # nicer layout
)


dotenv was True


/home/elba_ro/repos/github/conf22-style-transfer/.venv/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/elba_ro/repos/github/conf22-style-transfer/.venv/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


# Running Significance tests on Training sets

In [2]:
#significance[_key][0]


def filter_significance(significance_report,
                        col_name,
                        threshold: int = 0.00):
    def _apply_absolute_effect(row):
        effect = row[col_name] 
        row['absolute_effect'] = effect*(-1) if effect < 0.0 else effect
        return row
    _df : pd.DataFrame = significance_report.reset_index().copy()
    _df = _df[
        (~_df["feature"].str.endswith("_count")) & 
        (~_df["feature"].str.startswith("empath"))

    ]


    if "Unnamed: 0" in _df.columns.tolist():
        del _df["Unnamed: 0"]
    _df = _df.apply(_apply_absolute_effect, axis=1)
    _df = _df.sort_values(by=['absolute_effect'], ascending=False)
    _df = _df[_df['absolute_effect']>=threshold] if threshold>0.0 else _df
    _df.set_index("feature", inplace=True)

    return _df

import iesta.properties as prop

def calculate_filter_join_ideologies():
    ## GET DATA
    print("1/4 Getting data...")
    feature_dfs= {}
    _, liberal_df = iesta.machine_learning.dataloader.load_training_features_df("Liberal")
    feature_dfs["liberal"] = liberal_df

    _, conservative_df = iesta.machine_learning.dataloader.load_training_features_df("Conservative")
    feature_dfs["conservative"] = conservative_df

    ## CALCULATE SIGNIFICANCE
    print("2/4 Getting data...")
    significance_dict = iesta.stats.significance.run_all_significance_test(feature_dfs)

    # Filter Results based on the EFFECT Threshold
    print("3/4 Filtering data based in effect threshold...")


    # JOIN Results for LIBERAL AND CONSERVATIVE
    print("4/4 and Joining ideologies")
    joined_sign_dict = {}
    keys = [x for x in significance_dict.keys() if x.startswith("liberal")]
    for k in keys:
        joint_sign = pd.DataFrame()
        print(k)
        based_key = "_".join(k.split("_")[1:])
        print(based_key)
        liberal_df = significance_dict[f"liberal_{based_key}"][0].copy()
        conservative_df = significance_dict[f"conservative_{based_key}"][0].copy()
        col_name = [x for x in liberal_df.columns if "effective" in x][0]

        liberal_sign_df =  filter_significance(liberal_df, col_name, threshold=0.001)
        conservative_sign_df =  filter_significance(conservative_df, col_name, threshold=0.001)

        joint_sign = liberal_sign_df.join(conservative_sign_df, how='outer', lsuffix="_liberal", rsuffix="_conservative")

        def _apply_ideology_effect(row):
            is_liberal =  not pd.isna(float(row['absolute_effect_liberal'])) 
            is_cons =  not pd.isna(float(row['absolute_effect_conservative'])) 

            if is_liberal and is_cons:
                if row[f'{col_name}_liberal'] *  row[f'{col_name}_conservative'] > 0.0:
                    row['label'] = 'same'
                else: row['label'] = 'opposite'
            else: 
                row['label'] = 'conservative' if pd.isna(float(row['absolute_effect_liberal'])) else 'liberal'
            return row

        def _add_feature_type(row):
            row['feature_type'] = row.name.split("_")[0]
            return row
        joint_sign = joint_sign.apply(_apply_ideology_effect, axis = 1)
        joint_sign = joint_sign.sort_values(by=["label", "absolute_effect_conservative", "absolute_effect_liberal"])
        joint_sign = joint_sign.apply(_add_feature_type, axis= 1)
        joined_sign_dict[based_key] = joint_sign
        joint_sign.to_csv(f"../data/significant_test/summary_training_{based_key}.csv")
    return joined_sign_dict

#significance[k][0]

In [ ]:
significance_results = calculate_filter_join_ideologies()

In [ ]:
df = significance_results["effect"]
cross_df = pd.crosstab(df[ "label"], df["feature_type"])
cross_df['Total'] = cross_df.sum(axis=1)
cross_df.loc['Total']= cross_df.sum()
cross_df

In [ ]:
df.sort_values(by=["effective ineffective_conservative"], ascending = False)

## RUN Dim-Reduction

In [ ]:
import pandas as pd

In [3]:
ideology = "conservative"

sign_features_df = pd.read_csv("../data/significant_test/summary_training_effect.csv")
sign_features_df = sign_features_df[(sign_features_df["label"] == ideology) | (sign_features_df["label"] == "same") | (sign_features_df["label"] == "opposite")]
col = f"absolute_effect_{ideology}"
sign_features_df = sign_features_df.sort_values(by=[col], ascending=False)
sign_features_df[["feature", f"effective ineffective_{ideology}", "label"]]


,feature,effective ineffective_conservative,label
105,hedge_I_ratio,0.09,same
32,hedge_E_ratio,-0.07,opposite
104,liwc_OtherP,0.05,same
102,liwc_WC,0.05,same
103,hedge_D_ratio,0.05,same
...,...,...,...
43,liwc_Culture,0.01,same
42,liwc_Clout,-0.01,same
41,liwc_Analytic,0.01,same
40,liwc_want,0.01,same


In [27]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler, MaxAbsScaler, PowerTransformer, QuantileTransformer, Normalizer

def get_top_features(ideology: str):
    sign_features_df = pd.read_csv("../data/significant_test/summary_training_effect.csv")
    sign_features_df = sign_features_df[(sign_features_df["label"] == ideology) | (sign_features_df["label"] == "same") | (sign_features_df["label"] == "opposite")]
    col = f"absolute_effect_{ideology}"
    sign_features_df = sign_features_df.sort_values(by=[col], ascending=False)
    return sign_features_df[["feature", f"effective ineffective_{ideology}", "label"]]

def _add_score_features(row, ideology, top_features):
    feature_col_name = f"effective ineffective_{ideology}"
    features_score: float = 0.0
    for _, feature_row in top_features.iterrows():
        feature_name = feature_row['feature']
        features_score = features_score +  (row[feature_name] * feature_row[feature_col_name] * 1.0)
    row['features_score'] = features_score
    return row


def score_training_features(ideology: str) -> pd.DataFrame:
     
    top_features: pd.DataFrame = get_top_features(ideology)
    _, data = iesta.machine_learning.dataloader.load_training_features_df(ideology.capitalize())

    scaler = QuantileTransformer()
    columns_to_normalize = top_features["feature"].values.tolist()
    data[columns_to_normalize] = data[columns_to_normalize].fillna(0.0)
    data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
    
    data = data.apply(_add_score_features, axis=1, args=(ideology, top_features,))

    data = data.sort_values(by=['features_score'], ascending=False)
    return data

def generate_training_w_scores(ideology:str = "liberal", strategy: str = "feature-based"):
    if strategy == "feature-based":
        return score_training_features(ideology)
        


In [28]:
liberal_w_feature_score = generate_training_w_scores(ideology = "liberal", strategy="feature-based")

iesta        INFO     File already created. Loading file...


2135: ../data/extracted_features//Liberal_style-features_5000/Liberal_batch5000_6_style-features.parquet
35: ../data/extracted_features//Liberal_transformer-features_100/Liberal_batch100_272_transformer-features.parquet
excluding very long texts


In [29]:
liberal_w_feature_score

,id,debate_id,p_name,effect,category,round,argument,cleaned_text,is_for_eval_classifier,split,...,liwc_filler,liwc_AllPunc,liwc_Period,liwc_Comma,liwc_QMark,liwc_Exclam,liwc_Apostro,liwc_OtherP,liwc_Emoji,features_score
idx,,,,,,,,,,,,,,,,,,,,,
43715,The-patriots-are-the-greatest-football-team-in...,The-patriots-are-the-greatest-football-team-in...,bonnieboy,ineffective,Sports,2,\n \r\nyou know what. i probibly wont win thi...,"you know what. i probibly wont win this one, b...",False,training,...,0.0,1662.708119,-1.118557,0.546512,0.00,16187.50,-0.454955,-0.487768,0.0,246.149877
16342,Hillary-clinton-is-an-allien/1/|UtherPenguin,Hillary-clinton-is-an-allien/1/,UtherPenguin,ineffective,Politics,1,\n \r\nLies! Dear Leader would never laugh at...,Lies! Dear Leader would never laugh at his peo...,False,training,...,0.0,555.614594,-1.654639,-1.068475,52.38,5365.48,1.153153,-0.123853,0.0,81.830247
23920,MU-is-better-than-KU-football/1/|Funkymonkey,MU-is-better-than-KU-football/1/,Funkymonkey,ineffective,Sports,3,"\n \r\nI don't agree, YOUR A LIAR! MU IS THE ...","I don't agree, YOUR A LIAR! MU IS THE BEST!!!!...",False,training,...,0.0,459.936280,-1.654639,0.776486,0.00,4478.57,2.761261,-0.487768,0.0,67.638657
17928,I-will-not-break-a-rule./3/|wjmelements,I-will-not-break-a-rule./3/,wjmelements,ineffective,Miscellaneous,1,\n \r\n(Rule 4 Interjection: My opponent has ...,(Rule 4 Interjection: My opponent has broken a...,False,training,...,0.0,361.789311,1.466495,-0.117571,412.63,0.00,0.256757,474.724771,0.0,47.161767
17929,I-will-not-break-a-rule./3/|wjmelements,I-will-not-break-a-rule./3/,wjmelements,ineffective,Miscellaneous,2,\n \r\n(Rule 4 Interjection: It was impossibl...,(Rule 4 Interjection: It was impossible to fol...,False,training,...,0.0,295.781089,1.270619,-0.277778,0.00,0.00,0.333333,439.799694,0.0,41.788858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50366,can-be-not/1/|vi_spex,can-be-not/1/,vi_spex,ineffective,Science,0,\n \r\nis is,is is,False,training,...,0.0,-1.817061,-1.654639,-1.068475,0.00,0.00,-0.454955,-0.487768,0.0,-1.728388
48556,Whos-is-better-Lebron-James-or-Kobe-Bryant/1/|...,Whos-is-better-Lebron-James-or-Kobe-Bryant/1/,8803,ineffective,Sports,4,\n \r\nya they have,ya they have,False,training,...,0.0,-1.817061,-1.654639,-1.068475,0.00,0.00,-0.454955,-0.487768,0.0,-1.738436
51684,gun-control/47/|dairygirl4u2c,gun-control/47/,dairygirl4u2c,ineffective,Politics,4,\n \r\nya,ya,False,training,...,0.0,-1.817061,-1.654639,-1.068475,0.00,0.00,-0.454955,-0.487768,0.0,-1.783847


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_box(_df):
    #_df = liberal_w_feature_score#[liberal_w_feature_score["features_score"]< 1.5]
    # Assuming you already have the DataFrame _df with the columns 'features_score' and 'effect'

    # Use the groupby function to group the data by the 'effect' column
    grouped_data = _df.groupby('effect')['features_score']

    # Convert the grouped data into a list of lists to pass to the boxplot function
    data_to_plot = [group.values.tolist() for name, group in grouped_data]

    # Create the boxplot
    plt.figure(figsize=(8, 6))
    plt.boxplot(data_to_plot, labels=grouped_data.groups.keys())
    plt.xlabel('Effect')
    plt.ylabel('Features Score')
    plt.title('Boxplot of Features Score for each Effect')
    plt.grid(True)
    plt.show()

import seaborn as sns


def plot_box1(_df):
    
    # Set the figure size for the plot (optional but can be useful)
    plt.figure(figsize=(10, 6))

    # Use Seaborn's boxplot to plot 'features_score' for each 'effect'
    sns.boxplot(x='effect', y='features_score', data=_df)

    # Add numbers on the plot
    ax = sns.stripplot(x='effect', y='features_score', data=_df, color="black", size=4, jitter=True)

    # Set plot labels
    plt.xlabel('Effect', fontsize=12)
    plt.ylabel('Features Score', fontsize=12)
    plt.title('Boxplot of Features Score for each Effect', fontsize=14)

    # Show the plot
    plt.show()

In [30]:
cons_w_feature_score = generate_training_w_scores(ideology = "conservative", strategy="feature-based")

iesta        INFO     File already created. Loading file...


547: ../data/extracted_features//Conservative_style-features_5000/Conservative_batch5000_9_style-features.parquet
47: ../data/extracted_features//Conservative_transformer-features_100/Conservative_batch100_406_transformer-features.parquet
excluding very long texts


In [33]:

def save(df_w_score, ideology):
    for effect in ["effective", "ineffective"]:
        df = df_w_score[df_w_score["effect"] == effect]
        df = df[["effect", "cleaned_text", "features_score"]]
        ascending = effect == "ineffective"
        print(f"Saving for {effect} with ascending {ascending}")
        df = df.sort_values(by=["features_score"], ascending=ascending)
        df.to_parquet(f"../data/training/strategy_feature_based/{ideology}_{effect}.parquet")



In [34]:
pd.read_parquet(f"../data/training/strategy_feature_based/conservative_effective.parquet")

,effect,cleaned_text,features_score
idx,,,
22882,effective,.................................................,910.266436
61781,effective,:D k,9.394636
57723,effective,It's alright,7.879884
67885,effective,hweutihnjklsdhfjaklsdhfjklaytuioweytuwaeytsdhf...,7.655280
12662,effective,"Banter, Banter. :)",7.350190
...,...,...,...
70993,effective,You cannot say that humans cannot teleport bec...,-0.140849
68581,effective,Definition is there.,-0.144259
9395,effective,"Again, as the previous round was forfeited, I ...",-0.169886


In [32]:

save(liberal_w_feature_score, "liberal")

save(cons_w_feature_score, "conservative")

Saving for effective with ascending False
Saving for ineffective with ascending True
Saving for effective with ascending False
Saving for ineffective with ascending True


In [ ]:
get_top_features("conservative")['feature'].tolist()

In [ ]:
pip install umap-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from umap import UMAP
from sklearn.manifold import TSNE

%matplotlib inline

# PREPARE DATA FOR pca


# X_norm = (X - X.min())/(X.max() - X.min())
# X_standardized = StandardScaler().fit_transform(X)
def plot_reducer(df, ideology, reducer :str = "PCA", **kwargs):

    top_features = get_top_features(ideology)['feature'].tolist()
    X = df.loc[:, top_features].values

    # Apply Standard Scaling
    sc = StandardScaler()
    X = sc.fit_transform(X)
    # pd.DataFrame(X,columns=feature).head()
    # Define two components
    if reducer == "PSA":
        reducer = PCA(n_components=2)
    elif reducer == "UMAP":
        reducer = UMAP(n_components=2, **kwargs)
    elif reducer == "TSNE":
        reducer = TSNE(n_components=2)

    principalComponents = reducer.fit_transform(X)
    principalDf = pd.DataFrame(
        data=principalComponents,
        columns=["component 1", "component 2"],
    )
    # principalDf.head()
    finalDf = pd.concat([principalDf, df[["effect"]]], axis=1)

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel("Principal Component 1", fontsize=15)
    ax.set_ylabel("Principal Component 2", fontsize=15)
    ax.set_title(ideology, fontsize=20)
    targets = list(df.effect.unique())
    colors = ["b", "g", "y", "m"]  # sns.color_palette('deep')[:len(targets)]
    sorted_df = finalDf.sort_values(by=["component 1", "component 2"], ascending=False)
    top_examples= {}
    for target, color in zip(targets, colors):
        indicesToKeep = finalDf["effect"] == target

        
        top_examples[target] = sorted_df[sorted_df["effect"] == target]
        
        ax.scatter(
            finalDf.loc[indicesToKeep, "component 1"],
            finalDf.loc[indicesToKeep, "component 2"],
            c=color,
            s=50,
        )
        ax.xaxis.label.set_color("white")
        ax.yaxis.label.set_color("white")
        ax.title.set_color("white")
        ax.tick_params(axis="x", colors="white")
        ax.tick_params(axis="y", colors="white")
        ax.legend(targets)
        ax.grid()
    plt.figure(figsize=(16, 10))
    plt.show()

    return top_examples




In [ ]:
n_neighbors = [5, 10, 15, 20]
n_components = 2
metrics = ["euclidean", "manhattan", "cosine", "correlation", "mahalanobis", "jaccard"]


for metric, n_neighbor in [(x, y) for x in metrics for y in n_neighbors]:
    params = {"n_neighbors": n_neighbor,
               "metric": metric,}
    
    print(f"*** FOR n_neighbor {n_neighbor} and metric {metric}:")
    _ = plot_reducer(feature_dfs["liberal"], "liberal", "UMAP", **params)



In [ ]:
_ = plot_reducer(feature_dfs["liberal"], "liberal", "TSNE")

In [ ]:
_ = plot_reducer(feature_dfs["liberal"], "liberal", "UMAP")

In [ ]:
top_examples["ineffective"]